In [3]:
from collections import namedtuple

import numpy as np

## RSA example

Stolen from Will Monroe and Chris Pott's paper, [*Learning in the rational speech acts model.*][1]

[1]: http://nlp.stanford.edu/pubs/monroe2015learning.pdf

In [10]:
properties = ["beard", "glasses", "tie"]
Person = namedtuple("Person", properties)

In [6]:
def lexicon(word, referent):
    match = (   (word == "glasses" and referent.glasses)
             or (word == "beard" and referent.beard)
             or (word == "tie" and referent.tie))
    
    return 1.0 if match else 0.0

In [7]:
referents = [Person(glasses=True, beard=True, tie=False),
             Person(glasses=True, beard=False, tie=True),
             Person(glasses=False, beard=False, tie=True)]

## Literal speaker model

The literal speaker model $S_0$ is directly parameterized by the lexicon:

$$S_0(\text{word} \mid \text{referent}) \propto \exp(\lambda(\log \mathcal L(\text{word}, \text{referent}) - C(\text{word})))$$

In [13]:
literal_speaker = [[lexicon(word, referent) for word in properties]
                   for referent in referents]
literal_speaker = np.array(literal_speaker)
# Normalize.
literal_speaker /= literal_speaker.sum(axis=1, keepdims=True)

In [14]:
literal_speaker

array([[ 0.5,  0.5,  0. ],
       [ 0. ,  0.5,  0.5],
       [ 0. ,  0. ,  1. ]])

## Pragmatic listener model

The pragmatic listener maps then from word $\to$ referent:

$$L_1(\text{referent} \mid \text{word}) \propto S_0(\text{word} \mid \text{referent}) P(\text{referent})$$

In [15]:
pragmatic_listener = literal_speaker.T
# Normalize.
pragmatic_listener /= pragmatic_listener.sum(axis=1, keepdims=True)
pragmatic_listener

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.5       ,  0.5       ,  0.        ],
       [ 0.        ,  0.33333333,  0.66666667]])

## Pragmatic speaker model

The higher-level *pragmatic speaker* takes into account the mappings of the pragmatic listener.

$$S_1(\text{word} \mid \text{referent}) \propto \exp(\lambda(\log L_1(\text{referent} \mid \text{word}) - C(\text{word})))$$

In [16]:
pragmatic_speaker = pragmatic_listener.T
# Normalize.
pragmatic_speaker /= pragmatic_speaker.sum(axis=1, keepdims=True)
pragmatic_speaker

array([[ 0.66666667,  0.33333333,  0.        ],
       [ 0.        ,  0.6       ,  0.4       ],
       [ 0.        ,  0.        ,  1.        ]])